In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import plotly.graph_objects as go
from matplotlib.colors import Normalize, LinearSegmentedColormap
import matplotlib.cm as cm

# Data Preprocessing

In [2]:
# Load the dataset
# Replace with your dataset containing 'Year', 'Month', and 'Temperature Anomaly'
data = pd.read_csv('cleaned_data.csv')
data.head()

,Entity,Code,Year,Temperature anomaly,Population - Sex: all - Age: all - Variant: estimates,Annual CO₂ emissions (per capita),Day,Average monthly temperature,Average annual temperature
0,Afghanistan,AFG,1950,-2.298886,7776182,0.010837,1950-01-15,-2.655707,10.231125
1,Afghanistan,AFG,1950,-2.298886,7776182,0.010837,1950-02-15,-3.996040,10.231125
2,Afghanistan,AFG,1950,-2.298886,7776182,0.010837,1950-03-15,3.491112,10.231125
3,Afghanistan,AFG,1950,-2.298886,7776182,0.010837,1950-04-15,8.332797,10.231125
4,Afghanistan,AFG,1950,-2.298886,7776182,0.010837,1950-05-15,17.329062,10.231125


In [3]:
world_data = data[data["Entity"] == "World"]

In [4]:
world_data['Day'] = pd.to_datetime(world_data['Day'])
world_data = world_data.sort_values('Day')

# Prepare the data for yearly aggregation
world_data['Month'] = world_data['Day'].dt.month
world_data['Year'] = world_data['Day'].dt.year


/var/folders/pg/xznxg07956n15xkrst1hdvvm0000gn/T/ipykernel_84674/2243509737.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  world_data['Day'] = pd.to_datetime(world_data['Day'])


In [5]:
columns = ['Year', 'Month', 'Average monthly temperature']
world_data = world_data[columns].groupby(['Year', 'Month']).mean().reset_index()
world_data

,Year,Month,Average monthly temperature
0,1950,1,11.531446
1,1950,2,11.792894
2,1950,3,12.635241
3,1950,4,13.525023
4,1950,5,14.560052
...,...,...,...
883,2023,8,16.820910
884,2023,9,16.376972
885,2023,10,15.296179
886,2023,11,14.224238


In [6]:
world_data['Average monthly temperature'].min()

np.float64(11.523798)

In [7]:
world_data['Average monthly temperature'].max()

np.float64(16.953165)

# Visualization: Polar Plot

In [26]:
# Maps a normalized value (0 to 1) to a yellow-to-dark-red gradient.
def yellow_to_red(value):
    cmap = cm.get_cmap('YlOrRd')
    color = np.array(cmap(value)[:3])

    # Darken red by increasing red intensity while decreasing brightness in green
    red_boost_factor = 1.5  # Increase red dominance
    green_reduction_factor = 0.8  # Reduce green component to make red appear darker

    color[0] = min(1.0, color[0] * red_boost_factor)  # Boost red but cap at 1.0
    color[1] = color[1] * green_reduction_factor  # Reduce green to darken red
    color[2] = color[2] * 0.3  # Slightly reduce blue for a richer dark red

    return tuple(color)


In [33]:
# Ensure data is sorted by year and month
world_data = world_data.sort_values(by=['Year', 'Month'])

# Prepare the data for plotting
years = sorted(world_data['Year'].unique())
temperatures = world_data['Average monthly temperature']
norm = Normalize(vmin=11, vmax=17)  # Normalize temperature range for 11°C to 17°C

# Prepare traces
traces = []
month_labels = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

for i, year in enumerate(years):
    year_data = world_data[world_data['Year'] == year]
    r = year_data['Average monthly temperature'].values.clip(11, 17)  # Clip temperatures to range 11°C to 17°C
    theta = np.linspace(0, 2 * np.pi, 12, endpoint=False)  # Map months to angles
    theta_labels = month_labels  # Use month labels for angular positions

    # Map temperature values to colors using the orange-to-red gradient
    year_color = yellow_to_red(norm(r.mean()))  # Average color for the year

    # Add trace for the year (including the connection to the next year)
    traces.append(go.Scatterpolar(
        r=np.append(r, r[0]),  # Connect December to January
        theta=np.append(theta_labels, ['Jan']),  # Add January for closure
        mode='lines',
        hoverinfo='skip',
        line=dict(color=f'rgba({year_color[0] * 255}, {year_color[1] * 255}, {year_color[2] * 255}, 1)', width=2),
        showlegend=False  # Remove individual legend entries
    ))

# Add a slider to select the year range
steps = []
for i, year in enumerate(years):
    step = dict(
        method="update",
        args=[
            {"visible": [j <= i for j in range(len(traces))]},  # Show traces up to the selected year
            {"title": f"Polar Plot of Average Monthly Temperature (1950 Up to {year})"}
        ],
        label=str(year)
    )
    steps.append(step)

sliders = [dict(
    active=0,
    currentvalue={"prefix": "Year: "},
    pad={"t": 50},
    steps=steps
)]

# Create the layout
layout = go.Layout(
    title='Polar Plot of Average Monthly Temperature (1950-2023)',
    polar=dict(
        radialaxis=dict(
            range=[11, 17], 
            ticksuffix='°C',  # Append °C to tick labels
            tickvals=[11, 14, 17],  # Use 11, 14, and 17 directly as ticks
            ticktext=['11°C', '14°C', '17°C'],  # Simplify the tick labels
            showticklabels=True
        ),
        angularaxis=dict(
            direction='clockwise',
            tickmode='array',
            tickvals=list(range(12)),
            ticktext=month_labels  # Use month labels
        )
    ),
    sliders=sliders,
    dragmode=False,
    showlegend=False  # Remove the legend
)

# Set the first year trace as visible by default
if traces:
    traces[0]['visible'] = True

fig = go.Figure(data=traces, layout=layout)
fig.show()


/var/folders/pg/xznxg07956n15xkrst1hdvvm0000gn/T/ipykernel_84674/1431924824.py:3: MatplotlibDeprecationWarning:

The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.



# Export

In [15]:
fig.write_html("plotly_visualization.html")

# Reference
https://www.dataquest.io/blog/climate-temperature-spirals-python/